In [13]:
from datasets import load_dataset

dataset = load_dataset("Dahoas/instruct-synthetic-prompt-responses")
print(dataset["train"][0])



{'prompt': '  I was wondering if you could walk me through the process of setting up a hydroponic garden for herbs?\n\n \t\t\t ', 'response': "\nSure! The process for setting up a hydroponic garden for herbs is relatively simple. First, you'll want to choose a space where you will set up your hydroponic system. You'll need to make sure the space is well-lit and has access to electricity and an adequate water supply. Next, you'll need to choose the type of hydroponic system you want to use. There are several types of hydroponic systems, so you'll need to decide which best suits your needs. Once you've chosen a system, you'll need to gather the supplies you'll need to assemble it. This includes things like pumps, growing trays, grow lights, and nutrients. Once you've assembled the system, you'll need to add your choice of herbs to the system. Lastly, you'll need to monitor and adjust the system as needed to ensure your herbs are getting the correct amount of light, water, and nutrients."

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [23]:
def format_and_tokenize(example):
    full_prompt = f"### Instruction:\n{example['prompt']}\n\n### Response:\n{example['response']}"
    return tokenizer(full_prompt, truncation=True, padding="max_length", max_length=384)

tokenized_dataset = dataset["train"].map(format_and_tokenize, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/33203 [00:00<?, ? examples/s]

In [24]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./tinyllama-lora-out",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=2e-5,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    evaluation_strategy="no"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [21]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return at least 1


True
1


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()
model.save_pretrained("./tinyllama-lora-finetuned")
tokenizer.save_pretrained("./tinyllama-lora-finetuned")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2884\3688810459.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,1.787700
20,1.703300
30,1.730900
40,1.717700
50,1.682600
60,1.626200
70,1.577200
80,1.560100
90,1.479300
100,1.460500


('./tinyllama-lora-finetuned\\tokenizer_config.json',
 './tinyllama-lora-finetuned\\special_tokens_map.json',
 './tinyllama-lora-finetuned\\tokenizer.json')

In [26]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./tinyllama-lora-finetuned", tokenizer="./tinyllama-lora-finetuned", device=0)
prompt = "### Instruction:\nWhat is a neural network?\n\n### Response:\n"
result = pipe(prompt, max_new_tokens=100)
print(result[0]["generated_text"])


Device set to use cuda:0


### Instruction:
What is a neural network?

### Response:
A neural network is a type of artificial neural system that can be used to solve problems in various fields, including computer vision, natural language processing, and machine learning. It consists of a series of interconnected layers of neurons, each with its own set of weights and biases, that are used to process and analyze data. The network learns from experience and can be trained to perform tasks such as image recognition, speech recognition, and text classification.
